# Динамика Universe и Sample
Пример расчета численности целевых аудиторий в населении и выборки в динамике

## Описание задачи и условий расчета
- Период: 01.06.2022 - 31.08.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+, Все 12-24, Все 25-34, Все 35-44, Все 45+, Все 20-44, Все 20+, Мужчины, Женщины, Мужчины 20+, Женщины 20+, Мужчины 20-44, Женщины 20-44, Мужчины 35+, Женщины 35+
- Место просмотра: Все места (дом+дача)
- Каналы: ТНТ									
- Статистики: Universe, Sample

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [2]:
# выберем телесети по имени (части имени)
# для расчета показателей Universe и Sample в фильтр можно добавить любой канал
cats.get_tv_net(name=["ТНТ"])

# далее в запросах будем использовать id телесетей.

Запрошены записи: 0 - 0
Всего найдено записей: 0



""


In [3]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['пол','возраст'])

Запрошены записи: 0 - 129
Всего найдено записей: 129



,id,valueId,name,colName,valueName
0,1,1,Пол,Sex,Мужчины
1,1,2,Пол,Sex,Женщины
2,26,1,Семейное положение,MaritalStatus,Женат / замужем
3,26,2,Семейное положение,MaritalStatus,Холост / не замужем
4,2,33,Возраст,Age,33
...,...,...,...,...,...
124,2,10,Возраст,Age,10
125,2,89,Возраст,Age,89
126,2,8,Возраст,Age,8
127,18,0,Возраст детей: 2-3 года,KidsAge4,Нет


## Формирование задания
Зададим условия расчета

In [4]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-06-01', '2022-08-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId = 83'

# Указываем список статистик для расчета
statistics = ['Universe000', 'Sample']

# Указываем срезы
slices = ['researchMonth']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Формируем группу значений параметров для различных ЦА

In [5]:
# Задаем необходимые группы
targets = {
    'Все 4+':'age >= 4',
    '4-24':'age >= 4 AND age <= 24',
    '25-34':'age >= 25 AND age <= 34',
    '35-44':'age >= 35 AND age <= 44',
    '45+':'age >= 45',
    '20-44':'age >= 20 AND age <= 44',
    '20+':'age >= 20',
    'Мужчины ':'sex = 1',
    'Женщины':'sex = 2',
    'Мужчины 20+':'sex = 1 AND age >=20',
    'Женщины 20+':'sex = 2 AND age >=20',
    'Мужчины 20-44':'sex = 1 AND age >=20 AND age <= 44',
    'Женщины 20-44':'sex = 2 AND age >=20 AND age <= 44',
    'Мужчины 35+':'sex = 1 AND age >=35',
    'Женщины 35+':'sex = 2 AND age >=35'    
}

## Расчет задания

In [6]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждой ЦА формируем задания и отправляем на расчет
for target, logical in targets.items():
    
    # Формируем задание на расчет ЦА
    project_name = f"{target}" 
    basedemo_filter = f"{logical}"
      
    # Формируем из указанных параметров задание для TVI API в формате JSON для каждого из этапов цикла
    task_json = mtask.build_task('timeband', task_name=project_name, date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df = pd.concat(results)

Отправляем задания на расчет
...............
Ждем выполнения
Расчет задач (15) [ = ] время расчета: 0:00:04.294147
Расчет завершен, получаем результат
Собираем таблицу
...............

In [7]:
# округляем семпл до целого
df['Sample'] = df['Sample'].round(0)
df = df.astype({'Sample': 'int'})

In [8]:
df

,prj_name,researchMonth,Universe000,Sample
0,Все 4+,2022-06-01,136595.065989,17168
1,Все 4+,2022-07-01,136595.065981,16811
2,Все 4+,2022-08-01,136595.065986,16910
0,4-24,2022-06-01,32955.030554,4396
1,4-24,2022-07-01,32952.751720,4273
2,4-24,2022-08-01,32952.913037,4349
0,25-34,2022-06-01,20503.966278,2091
1,25-34,2022-07-01,20505.454131,2006
2,25-34,2022-08-01,20505.242932,1994
0,35-44,2022-06-01,22485.840366,3584


## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [9]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('04_universe&sample_by_months'))
df_info = mtask.task_builder.get_report_info()
df.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()